# Solving the Knapsack Problem with Quantum
In this notebook we take on the challenge of solving a common constraint satisfaction problem known as the knapsack problem.

* Hardware: D-Wave Advantage System version 1.1
* Provider: Amazon Web Services Braket

notebook by [Brian Lechthaler](https://github.com/brianlechthaler) | gnu gpl v3 | 2020-12-15 | please don't distribute without explicit written consent and/or giving attribution

## Part 1: Introduction
### 1.0: Shipping Fruit
Let's imagine for a moment that we are farmers who export their fruit to other countries. We have 4 fruits to ship abroad: grapes, apples, bananas and pears. Each item, as shown in the figure below, is packaged in a large box to then be packed into a standard shipping container.

<img src='https://storage.googleapis.com/jupyter_assets-463b/knapsack_fruits.svg'>

### 1.1 The Problem
There's plenty of room in the shipping container to fit as many boxes as we need, but due to the weight limits of shipping containers and the vehicles used to transport them we can only store so many boxes of each fruit before hitting weight limits. 

What's the most profitable way can we fill our container without exceeding weight limits while also exceeding weight limits?

<img src="https://storage.googleapis.com/jupyter_assets-463b/knapsack_shippingContainer.svg">

### 1.2: The Solution
This problem that we are faced with is characteristic of the exact kind of problem Quantum Annealers excel at: it's a constraint solution problem that can be expressed as a Binary Quadratic Model, or BQM for short. 

### 1.3: Technical Overview of Solution
This notebook starts by generating a BQM from input variables. Then, a sampler and EmbeddingComposite() is created for our D-Wave machine on AWS. We then send a request over to our D-Wave machine that queues our BQM as a new job and runs it as soon as there is availibility. Once our BQM is on the QPU, it gets solved very quickly (nanoseconds to milliseconds) and returned back to us. Even after we test the limits and scale until we can't anymore, it still only takes a fraction of a fraction of a second to run. This is the magic of quantum: solving large scale equations with many variables in quadratic time, as opposed to linear or even logarithmic time for a classical computer performing the same operation.

## Part 2: Imports and Configuration

In [ ]:
# Pull the original example to get the datasets. Comment this out if you don't need it.
!git clone https://github.com/dwave-examples/knapsack.git

In [1]:
!pip install -qq autopep8
!pip install -qq jupyter_contrib_nbextensions==0.5.1
!pip install -qq bokeh==0.12.15

In [2]:
import itertools
from itertools import combinations
from dwave.system.composites import EmbeddingComposite
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler
import time
from datetime import datetime as dt
from helpers.embedding import embeddings
import pandas as pd
import dimod
from math import log2, floor
from bucket import get_bucket as bucket
import draw

Loading BokehJS ...

In [3]:
# notebook specific config
dwave_qpu = 'Advantage_system1'
notebook_start_time = time.time()

In [4]:
#AWS specific config
# Please enter the S3 bucket you created during onboarding in the code below
my_bucket = bucket() # the name of the bucket
my_prefix = "results" # the name of the folder in the bucket
s3_folder = (my_bucket, my_prefix)


## Part 3: Defining Functions
### 3.0: Timestamper
This is a helpful function to quickly add a timestamp to the output of this notebook.

In [5]:
def tsmsg(msg=None):
    now = dt.now()
    timestamp = f'[@{str(now)}]: '
    message = timestamp + msg
    return message

### 3.1: Sampler Builder
This function creates our sampler along with an EmbeddingComposite() for the sampler.

In [6]:
def mksampler():
    arn = 'arn:aws:braket:::device/qpu/d-wave/' + dwave_qpu
    print(tsmsg(f'Building sampler from {arn}...'))
    sampler = BraketDWaveSampler(s3_folder, arn)
    print(tsmsg(f'BraketDWaveSampler successfully built for {dwave_qpu} on AWS Braket.\n    -> Building EmbeddingComposite for sampler...'))
    sampler = EmbeddingComposite(sampler)
    print(tsmsg(f'EmbeddingComposite() successfully built.\n    -> Returning sampler.'))
    return sampler

### 3.2: Sampler Runner
This function is responsible for sending the BQM to the D-Wave machine for solving.

In [7]:
def runSampler(bqm):
    x = '\n    -> '
    print(tsmsg('Building sampler...'))
    sampler = mksampler()
    print(tsmsg(f"[CLASSIC->QUANTUM] Creating a job to anneal on {dwave_qpu}'s QPU, and awaiting results..."))
    sampStart = time.time()
    sampleset = sampler.sample(bqm)
    sampEnd = time.time()
    sampDelta = sampEnd - sampStart
    print(tsmsg(f"[QUANTUM->CLASSIC] Annealing completed successfully.{x}{sampDelta} seconds have elapsed since we sent our job to {dwave_qpu}.{x}Returning results from from {dwave_qpu}'s QPU."))
    sample = sampleset.first.sample
    energy = sampleset.first.energy
    return sampleset, sample, energy

### 3.3: Knapsack Builder
This function converts our costs, weights, and the maximum weight we can carry into a BQM.

In [8]:
def build_knapsack_bqm(costs, weights, weight_capacity):
    x = '\n    -> '
    """Construct BQM for the knapsack problem
    
    Args:
        costs (array-like):
            Array of costs associated with the items
        weights (array-like):
            Array of weights associated with the items
        weight_capacity (int):
            Maximum allowable weight
    
    Returns:
        Binary quadratic model instance
    """
    print(tsmsg('Building BQM.'))
    buildbqmstart = time.time()
    # Initialize BQM - use large-capacity BQM so that the problem can be
    # scaled by the user.
    bqm = dimod.AdjVectorBQM(dimod.Vartype.BINARY)

    # Lagrangian multiplier
    # First guess as suggested in Lucas's paper
    lagrange = max(costs)

    # Number of objects
    x_size = len(costs)

    # Lucas's algorithm introduces additional slack variables to
    # handle the inequality. M+1 binary slack variables are needed to
    # represent the sum using a set of powers of 2.
    M = floor(log2(weight_capacity))
    num_slack_variables = M + 1

    # Slack variable list for Lucas's algorithm. The last variable has
    # a special value because it terminates the sequence.
    y = [2**n for n in range(M)]
    y.append(weight_capacity + 1 - 2**M)

    # Hamiltonian xi-xi terms
    for k in range(x_size):
        bqm.set_linear('x' + str(k), lagrange * (weights[k]**2) - costs[k])

    # Hamiltonian xi-xj terms
    for i in range(x_size):
        for j in range(i + 1, x_size):
            key = ('x' + str(i), 'x' + str(j))
            bqm.quadratic[key] = 2 * lagrange * weights[i] * weights[j]

    # Hamiltonian y-y terms
    for k in range(num_slack_variables):
        bqm.set_linear('y' + str(k), lagrange * (y[k]**2))

    # Hamiltonian yi-yj terms
    for i in range(num_slack_variables):
        for j in range(i + 1, num_slack_variables):
            key = ('y' + str(i), 'y' + str(j))
            bqm.quadratic[key] = 2 * lagrange * y[i] * y[j]

    # Hamiltonian x-y terms
    for i in range(x_size):
        for j in range(num_slack_variables):
            key = ('x' + str(i), 'y' + str(j))
            bqm.quadratic[key] = -2 * lagrange * weights[i] * y[j]
    buildbqmend = time.time()
    buildbqmdelta = buildbqmend - buildbqmstart
    print(tsmsg(f'Done.{x} BQM built in {buildbqmdelta} seconds.'))
    return bqm

### 3.4: Compile the BQM
This function uses the logic we just defined to create a BQM to be solved on our D-Wave machine, and time how long it takes to do so.

In [9]:
def compileBQM(costs, weights, weight_capacity):
    #Build knapsack problem from costs, weights, and max capacity
    x = '\n    -> '
    print(tsmsg(f"Compiling BQM with: {x}Input Costs: {costs}{x}Weights: {weights}{x}Max Weight: {weight_capacity}"))
    bqmstart = time.time()
    bqm = build_knapsack_bqm(costs, weights, weight_capacity)
    bqmend = time.time()
    bqmdelta = bqmend - bqmstart
    print(tsmsg(f"BQM compiled in {bqmdelta} seconds."))
    return bqm

### 3.5: Solve BQM
Now that we have our BQM all we have to do is send it to our D-Wave quantum annealer and wait for the response, then we decide whether the result is valid, run again if it isn't and finally return the valid solution when we are done. Note the valid solution verification step at the end, this kind of error handling is vital to building good quantum solutions: because at the end of the day we're relying on quantum physics to run calculations for production workloads.

In [10]:
def solve_knapsack():
    return None
def solve_knapsack(costs, weights, weight_capacity, sampler=None, xdf=pd.DataFrame()):
    """Construct BQM and solve the knapsack problem
    
    Args:
        costs (array-like):
            Array of costs associated with the items
        weights (array-like):
            Array of weights associated with the items
        weight_capacity (int):
            Maximum allowable weight
        sampler (BQM sampler instance or None):
            A BQM sampler instance or None, in which case
            LeapHybridSampler is used by default
    
    Returns:
        Tuple:
            List of indices of selected items
            Solution energy
    """
    original_costs = costs
    original_weights = weights
    original_weightcap = weight_capacity
    # compile a BQM from input variables
    bqm = compileBQM(costs, weights, weight_capacity)
    
    # run the sampler and read the results into 3 variables
    sampleset, sample, energy = runSampler(bqm)

    # Build solution from returned binary variables:
    selected_item_indices = []
    for varname, value in sample.items():
        # For each "x" variable, check whether its value is set, which
        # indicates that the corresponding item is included in the
        # knapsack
        if value and varname.startswith('x'):
            # The index into the weight array is retrieved from the
            # variable name
            selected_item_indices.append(int(varname[1:]))
    selected_weights = list(xdf.loc[selected_item_indices, 'weight'])
    total_weight = sum(selected_weights)
    if total_weight > original_weightcap:
        print(tsmsg('ERROR: Invalid solution, total weight exceeds specified capacity. Attempting to try again.'))
        solve_knapsack(original_costs, original_weights, original_weightcap, sampler, xdf=xdf)
    else:
        print(tsmsg(''))
    return sorted(selected_item_indices), energy

### 3.6: Tying it all Together (Main Function)
Finally, we take all the pieces we just defined and put them together to make our main function. 

In [11]:
def knapsackSolver(wt_cap=None, pth=None):
    
    # set defaults just in case
    if wt_cap == None:
        wt_cap = 50
    if pth == None:
        pth = 'knapsack/data/small.csv'

    # parse input data
    print(tsmsg(f'Reading the contents of file {pth} into a new Pandas DataFrame.'))
    csvreadstart = time.time()
    df = pd.read_csv(pth, names=['cost', 'weight'])
    csvreadstop = time.time()
    csvreaddelta = csvreadstop - csvreadstart
    print(tsmsg(f'New dataframe created from file {pth} in {csvreaddelta} seconds.'))

    selected_item_indices, energy = solve_knapsack(df['cost'], df['weight'], wt_cap, xdf=df)
    selected_weights = list(df.loc[selected_item_indices,'weight'])
    selected_costs = list(df.loc[selected_item_indices,'cost'])
    energysummary = "----> SOLUTION: Found solution at energy {}".format(energy)
    print(tsmsg(energysummary))
    itemsummary = "----> SOLUTION: Selected item numbers (0-indexed): " + str(selected_item_indices)
    print(tsmsg(itemsummary))
    weightsummary = "----> SOLUTION: Selected item weights: {}, total = {}".format(selected_weights, sum(selected_weights))
    print(tsmsg(weightsummary))
    costsummary = "----> SOLUTION: Selected item costs: {}, total = {}".format(selected_costs, sum(selected_costs))
    print(tsmsg(costsummary))

## Part 4: Testing & Results!
### 4.0: First Test
For our first test run, we try a max weight of 50 with the "small" dataset. As expected, the D-Wave Annealer has absolutely no problem solving this problem in nanoseconds-to-milliseconds.

In [12]:
knapsackSolver(50, 'knapsack/data/small.csv')

[@2020-12-16 00:21:55.396850]: Reading the contents of file knapsack/data/small.csv into a new Pandas DataFrame.
[@2020-12-16 00:21:55.401228]: New dataframe created from file knapsack/data/small.csv in 0.0038251876831054688 seconds.
[@2020-12-16 00:21:55.402040]: Compiling BQM with: 
    -> Input Costs: 0    35
1    85
2    30
3    50
4    70
5    80
6    55
Name: cost, dtype: int64
    -> Weights: 0    12
1    27
2    11
3    17
4    20
5    10
6    15
Name: weight, dtype: int64
    -> Max Weight: 50
[@2020-12-16 00:21:55.402095]: Building BQM.
[@2020-12-16 00:21:55.402802]: Done.
    ->  BQM built in 0.0006716251373291016 seconds.
[@2020-12-16 00:21:55.402845]: BQM compiled in 0.0007488727569580078 seconds.
[@2020-12-16 00:21:55.402875]: Building sampler...
[@2020-12-16 00:21:55.402907]: Building sampler from arn:aws:braket:::device/qpu/d-wave/Advantage_system1...
[@2020-12-16 00:21:57.475594]: BraketDWaveSampler successfully built for Advantage_system1 on AWS Braket.
    -> Buildin

### 4.1: Second Test
Yet again, our annealer proves what can only be described as quantum's *complete and total ascendancy* over any model or practical example we have to describe even our wildest dreams of classical computer performance. Just imagine what the world will be able to do once we can eliminate current barriers like noise(interference), size, temperature (superconductors currently used in quantum computers need to be kept at well over 150x colder temperatures than interstellar space.)

In [13]:
knapsackSolver(75, 'knapsack/data/small.csv')

[@2020-12-16 00:22:02.982601]: Reading the contents of file knapsack/data/small.csv into a new Pandas DataFrame.
[@2020-12-16 00:22:02.984578]: New dataframe created from file knapsack/data/small.csv in 0.0017132759094238281 seconds.
[@2020-12-16 00:22:02.985235]: Compiling BQM with: 
    -> Input Costs: 0    35
1    85
2    30
3    50
4    70
5    80
6    55
Name: cost, dtype: int64
    -> Weights: 0    12
1    27
2    11
3    17
4    20
5    10
6    15
Name: weight, dtype: int64
    -> Max Weight: 75
[@2020-12-16 00:22:02.985287]: Building BQM.
[@2020-12-16 00:22:02.985999]: Done.
    ->  BQM built in 0.0006711483001708984 seconds.
[@2020-12-16 00:22:02.986038]: BQM compiled in 0.0007493495941162109 seconds.
[@2020-12-16 00:22:02.986071]: Building sampler...
[@2020-12-16 00:22:02.986098]: Building sampler from arn:aws:braket:::device/qpu/d-wave/Advantage_system1...
[@2020-12-16 00:22:04.937097]: BraketDWaveSampler successfully built for Advantage_system1 on AWS Braket.
    -> Buildin

### 4.1 Final Tests
Sometimes when giving our annealer a problem that pushes it's limits in terms of how large the solvable problem space we encounter errors. Try scaling the datasets and see what the largest combination you can achieve is.

In [14]:
knapsackSolver(150, 'knapsack/data/small.csv')

[@2020-12-16 00:22:08.923672]: Reading the contents of file knapsack/data/small.csv into a new Pandas DataFrame.
[@2020-12-16 00:22:08.929194]: New dataframe created from file knapsack/data/small.csv in 0.0049211978912353516 seconds.
[@2020-12-16 00:22:08.929910]: Compiling BQM with: 
    -> Input Costs: 0    35
1    85
2    30
3    50
4    70
5    80
6    55
Name: cost, dtype: int64
    -> Weights: 0    12
1    27
2    11
3    17
4    20
5    10
6    15
Name: weight, dtype: int64
    -> Max Weight: 150
[@2020-12-16 00:22:08.929959]: Building BQM.
[@2020-12-16 00:22:08.931521]: Done.
    ->  BQM built in 0.0015225410461425781 seconds.
[@2020-12-16 00:22:08.931581]: BQM compiled in 0.0016188621520996094 seconds.
[@2020-12-16 00:22:08.931611]: Building sampler...
[@2020-12-16 00:22:08.931640]: Building sampler from arn:aws:braket:::device/qpu/d-wave/Advantage_system1...
[@2020-12-16 00:22:10.932700]: BraketDWaveSampler successfully built for Advantage_system1 on AWS Braket.
    -> Buildi

### Part 5: Conclusions
The knapsack problem, as with all constraint-satisfaction problems can be turned into a BQM. With our D-Wave quantum annealer, we can basically solve anything that is a BQM. By turning our problem into a BQM and solving it, we effectively exploit the dramatic speedup quantum offers for solving CSPs with large numbers of variables. The results speak for themselves. As we scale the problem up and down there are admittedly some errors we run into that are related to the actual accuracy of the machine. Over time, as quantum becomes less noisy and prone to errors, quantum computers could solve a wide range of problems at almost any scale in very small amounts of time. This massive boost in speed has the potential to usher in a new era of computing that brings use cases and solutions we could have never thought of even in our most optimistic dreams. Will they replace classical computers? Likely not, each has their advantages and weaknesses -- one cannot replace the other if one cannot be the other. Will quantum allow us to perform calculations we could not have before? Definitely. The speedup isn't just more convenient -- it makes easy work of certain problems like large prime factorisation previously thought to be impossible even for some future civilization that uses most of the universe as a classical computer to solve. 

In [15]:
notebook_end_time = time.time()
notebook_time_delta = notebook_end_time - notebook_start_time
print(tsmsg(f'Once dependencies were done importing, this notebook ran for a grand total of {notebook_time_delta} seconds.\n    Notebook by Brian Lechthaler, last updated: {str(dt.now())}.\n    Licensed under GNU GPL v3.\n    Do not re-distribute notebook or any of its contents without my explicit written consent and/or without attribution.\n    This notebook belongs to this repository: https://github.com/brianlechthaler/QuantumSolutions\n        If you found this useful, leave a star. Problems? Questions? Contact me or submit a pull request and I will be delighted to help you.'))

[@2020-12-16 00:22:17.073622]: Once dependencies were done importing, this notebook ran for a grand total of 21.7499098777771 seconds.
    Notebook by Brian Lechthaler, last updated: 2020-12-16 00:22:17.073613.
    Licensed under GNU GPL v3.
    Do not re-distribute notebook or any of its contents without my explicit written consent and/or without attribution.
    This notebook belongs to this repository: https://github.com/brianlechthaler/QuantumSolutions
        If you found this useful, leave a star. Problems? Questions? Contact me or submit a pull request and I will be delighted to help you.
